In [1]:
from SE3 import se3, SE3
from SO3 import so3, DCM, Euler, MRP, Quat
from SE2 import se2, SE2
from SO2 import so2, SO2
import casadi as ca
%load_ext autoreload
%autoreload 2

All inputs for group/algebra has to be np.array([]) (e.g., se3(x) where x is a np.array which has shape (6,1) or (6,))

Algebra:

- property: wedge, ad_matrix
- method: vee (input: Lie algebra matrix, output: vector of algebra element)

Group:

- property: to_matrix, inv, Ad_matrix
- method: 
    - to_vect (input: Lie group matrix, output: vector of Group element)
    - log (input: Group (e.g., SE3(x)), output: return Algebra) 
    - exp (input: Algebra (e.g., se3(x)), output: return Group)

** use wedge/to_matrix get the group/algebra to a matrix of np.array

** use .param get the element of group/algebra to a vector of np.array (e.g., x = se3(x).param)

** inputs for tranformation in SO3 should be np.array (e.g., DCM.from_euler(np.array), Euler.from_dcm(np.array))

** both @ and * can do multiplication for group, both matrix has to be in same group!

** @ still can do np array/matrix multiplication, as long as two elements are both in np

In [2]:
x = ca.DM([1,2,3,0.1,0.2,0.3])
y = ca.DM([2,4,6,0.01,0.02,0.03])
theta = x[3:6]

In [3]:
a = se3(x)
a

SX([1, 2, 3, 0.1, 0.2, 0.3])

In [4]:
a.param.shape

(6, 1)

In [5]:
#test so3
a = so3(theta)
aw = a.wedge
print('wedge:', aw)
av = so3.vee(aw)
print('vee:', av)

A = DCM.exp(a) # return group of DCM
print('exp:', A)
theta_group = Euler.from_dcm(A)
print('exp_ele:', theta_group)

wedge: 
[[00, -0.3, 0.2], 
 [0.3, 00, -0.1], 
 [-0.2, 0.1, 00]]
vee: [0.1, 0.2, 0.3]
exp: 
[[0.935755, -0.283165, 0.210192], 
 [0.302933, 0.950581, -0.0680313], 
 [-0.18054, 0.127335, 0.97529]]
exp_ele: [0.129826, 0.181536, 0.313084]


In [6]:
# test SO3
B = DCM.from_euler(theta_group) # group of DCM
print('SO3:', B)
print('inv', B.inv)
b = DCM.log(B) # input of log should be group, return so3 algebra (which will show element)
print('log:', b.wedge)
print('log_v:', b)


SO3: 
[[0.935755, -0.283165, 0.210192], 
 [0.302933, 0.950581, -0.0680313], 
 [-0.18054, 0.127335, 0.97529]]
inv 
[[0.935755, 0.302933, -0.18054], 
 [-0.283165, 0.950581, 0.127335], 
 [0.210192, -0.0680313, 0.97529]]
0.374166
log: 
[[00, -0.3, 0.2], 
 [0.3, 00, -0.1], 
 [-0.2, 0.1, 00]]
log_v: [0.1, 0.2, 0.3]


In [7]:
# check inv and matmul for group
DCM(B.inv)@B

SX(@1=1, @2=-5.55112e-17, @3=0, 
[[@1, @2, @3], 
 [@2, @1, @3], 
 [@3, @3, 1]])

In [8]:
# matmul can also be used in np.array
B.inv@B.to_matrix

SX(@1=1, @2=-5.55112e-17, @3=0, 
[[@1, @2, @3], 
 [@2, @1, @3], 
 [@3, @3, 1]])

In [12]:
# test se3
c = se3(x)
cw = c.wedge
print('element:', c)
print('wedge:', cw)
print('vee:', se3.vee(cw))

C = SE3.exp(c)
print('exp:', C.to_matrix)
print('exp_ele:', C)

element: [1, 2, 3, 0.1, 0.2, 0.3]
wedge: @1=0, 
[[00, -0.3, 0.2, 1], 
 [0.3, 00, -0.1, 2], 
 [-0.2, 0.1, 00, 3], 
 [@1, @1, @1, @1]]
vee: [1, 2, 3, 0.1, 0.2, 0.3]
exp: @1=0, @2=1, 
[[0.935755, -0.283165, 0.210192, @2], 
 [0.302933, 0.950581, -0.0680313, 2], 
 [-0.18054, 0.127335, 0.97529, 3], 
 [@1, @1, @1, @2]]
exp_ele: [1, 2, 3, 0.129826, 0.181536, 0.313084]


In [13]:
c.ad_matrix

SX(@1=0.3, @2=-0.2, @3=0, @4=-0.3, @5=0.1, @6=0.2, @7=-0.1, 
[[00, @4, @6, 00, -3, 2], 
 [@1, 00, @7, 3, 00, -1], 
 [@2, @5, 00, -2, 1, 00], 
 [@3, @3, @3, 00, @4, @6], 
 [@3, @3, @3, @1, 00, @7], 
 [@3, @3, @3, @2, @5, 00]])

Note that the elements of SE3 Lie group has to use [x,y,z,theta1,theta2,theta3] (pos, euler angles)

if you have your angle in other format, use SO3 class to do the transformation first

In [14]:
# test SE3
D = SE3(SE3.to_vec(C.to_matrix))
print('SE3:', D.to_matrix)
print('inv:', D.inv)
d = SE3.log(D) 
print('log:', d)

SE3: @1=0, @2=1, 
[[0.935755, -0.283165, 0.210192, @2], 
 [0.302933, 0.950581, -0.0680313, 2], 
 [-0.18054, 0.127335, 0.97529, 3], 
 [@1, @1, @1, @2]]
inv: @1=0, 
[[0.935755, 0.302933, -0.18054, -1], 
 [-0.283165, 0.950581, 0.127335, -2], 
 [0.210192, -0.0680313, 0.97529, -3], 
 [@1, @1, @1, 1]]
0.374166
log: [1, 2, 3, 0.1, 0.2, 0.3]


In [15]:
# check inv
D.to_matrix@D.inv

SX(@1=1, @2=6.245e-17, @3=-2.77556e-17, @4=0, @5=-1.38778e-17, 
[[@1, @2, @3, -4.44089e-16], 
 [@2, @1, @5, -6.66134e-16], 
 [@3, @5, 1, @4], 
 [@4, @4, @4, 1]])

$$ad(x)*y = - ad(y)*x$$

In [16]:
x1 = ca.DM([1,2,3,0.1,0.12,0.13])
y1 = ca.DM([3,4,5,0.3,0.2,0.34])
se3(x1).ad_matrix@y1 + se3(y1).ad_matrix@x1

SX(@1=0, [@1, 5.55112e-17, @1, @1, @1, @1])

In [17]:
x2 = ca.DM([1,2,0.1])
y2 = ca.DM([3,4,0.34])
se2(x2).ad_matrix@y2 + se2(y2).ad_matrix@x2

SX(@1=0, [@1, @1, 00])

$$Ad(\exp([x]^{\wedge})) = \exp(ad([x]^{\wedge})) $$

In [18]:
from scipy import linalg

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [22]:
ca.DM(SE3.exp(se3(x1)).Ad_matrix)

DM(
[[0.984404, -0.123128, 0.125653, -0.625936, -2.78917, 2.17065], 
 [0.135086, 0.986596, -0.0915399, 3.10661, -0.486684, -0.66091], 
 [-0.112698, 0.107086, 0.987842, -1.74371, 1.27689, -0.337351], 
 [0, 0, 0, 0.984404, -0.123128, 0.125653], 
 [0, 0, 0, 0.135086, 0.986596, -0.0915399], 
 [0, 0, 0, -0.112698, 0.107086, 0.987842]])

In [24]:
ca.DM(SE2.exp(se2(x2)).Ad_matrix)

DM(
[[0.995004, -0.0998334, 2.04663], 
 [0.0998334, 0.995004, -0.898417], 
 [0, 0, 1]])